In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

## Run this if you want to access models from google drive
import os
print(os.getcwd())
!ls

In [ ]:
!pip install -U --pre tensorflow=="2.10.1"

In [ ]:
import os
import pathlib
os.chdir('drive/My Drive/')
print(os.getcwd())

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Object detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
#run model builder test
!python /content/drive/"My Drive"/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
print(os.getcwd())
os.chdir('models')
print(os.getcwd())

In [ ]:
os.chdir('train')
print(os.getcwd())
#os.mkdir('train')
!ls

In [ ]:
print(os.getcwd())
os.chdir('/content/drive/My Drive/models')
print(os.getcwd())

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    classes_names = []

    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    classes_names = list(set(classes_names))
    classes_names.sort()

    return xml_df, classes_names

def main():
    for directory in ['train', 'test']:
        print(directory)
        image_path = os.path.join(os.getcwd(), 'research/object_detection/images/{}'.format(directory))
        xml_df, classes = xml_to_csv(image_path)
        print(os.getcwd())
        xml_df.to_csv('research/object_detection/data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')
    

main()

In [ ]:
# Convert csv to tf record files - To switch to a code based version so that the sys.exit dont take place.

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

"""
Usage:
  # From tensorflow/models/

  # Create train data:
  python Generate_TF_Records.py --csv_input=data/train_labels.csv  --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record

"""


#flags = tf.compat.v1.flags
#flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
#flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
#flags.DEFINE_string('image_dir', '', 'Path to images')
#LAGS = flags.FLAGS

# TO-DO replace this with label map

def class_text_to_int(row_label):
    if row_label == 'vinyl banner':
        return 1
    elif row_label == 'property sign':
        return 2
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    print(path)
    print(group.filename)
    with tf.compat.v1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example

def main(_):
    for directory in ['train', 'test']:
      output_path = os.path.join(os.getcwd(), 'research/object_detection/data/{}.record'.format(directory))
      print(output_path)
      input_path = os.path.join(os.getcwd(), 'research/object_detection/data/{}_labels.csv'.format(directory))
      print(input_path)
      writer = tf.io.TFRecordWriter(output_path)
      print(os.getcwd())
      path = os.path.join(os.getcwd(), 'research/object_detection/images/{}'.format(directory))
      print(path)
      examples = pd.read_csv(input_path)
      grouped = split(examples, 'filename')
      for group in grouped:
          tf_example = create_tf_example(group, path)
          writer.write(tf_example.SerializeToString())

      writer.close()
      #output_path = os.path.join(os.getcwd(), FLAGS.output_path)
      print('Successfully created the TFRecords: {}'.format(output_path))

if __name__ == '__main__':
    tf.compat.v1.app.run()
     

In [ ]:
os.chdir('/content/drive/My Drive/models/research/object_detection/images')
os.chdir('/content/drive/My Drive/models/research/object_detection')
print(os.getcwd())
labelmap_path = os.path.join(os.getcwd(), 'data/label_map.pbtxt')
train_record_path = os.path.join(os.getcwd(), 'data/train.record')
test_record_path = os.path.join(os.getcwd(), 'data/test.record')
print(test_record_path)
print(train_record_path)

In [ ]:
os.mkdir('pre-trained-models')
os.chdir('pre-trained-models')
os.mkdir('ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8')
os.chdir('ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8')
os.mkdir('checkpoint')
os.mkdir('saved_model')

In [ ]:
os.chdir('saved_model')
os.mkdir('variables')

In [ ]:
os.chdir('/content/drive/My Drive/models/research/object_detection')
fine_tune_checkpoint = 'pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0'
base_config_path = 'pre-trained-models/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config'

In [ ]:
print(os.getcwd())

# edit configuration file (from https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD)

batch_size = 12
num_steps = 50000
num_eval_steps = 1000

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(2), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model_config.config

In [ ]:
print(os.getcwd())
!ls
os.chdir('/content/drive/My Drive/models/research/object_detection')
model_dir = 'training/'
pipeline_config_path = 'model_config.config'

In [ ]:
os.chdir('/content/drive/My Drive/models/research/object_detection')
print(os.getcwd())
!ls

In [ ]:
batch_size = 12
num_steps = 50000
num_eval_steps = 1000
!python model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}